> https://www.kaggle.com/startupsci/titanic-data-science-solutions

# Titanic问题的数据挖掘步骤
## 首先介绍 Data Science Solutions书中处理的步骤


1. 问题定义
2. 构建训练集和测试集数据
3. 准备和清洗数据
4. 分析数据，识别特征模式
5. 建模，预测并解决问题
6. 可视化报告最终的方案
7. 提交结果


- 步骤可以打乱穿插

# 1. 问题定义

来自Kaggle竞赛Titanic的介绍：
> 已知训练集和测试集，训练集中是891个人的个人信息和是否存活，测试集中是个人信息，要求使用训练集训练我们的模型，判断测试集中的人是否存活

一些需要知道的背景知识：
- 1912年4月15日，Titanic首次航行撞冰山沉没2224个乘客中1505个丧生，存活率为32%
- 没有足够的救生船是死亡率这么高的一个原因
- 即使有一些幸运的成分，但是有些群体更容易存活，比如说女人、小孩和上层阶层

# 2. 解决流程
- **Classifying**：对样本进行归类，不同类之间的相关性进行分析
- **Correlating**：可以通过训练集现有的特征来解决问题。在这个数据集下，哪个特征对结果有贡献？从统计角度看特征和结果有什么相关性？改变特征值结果会不会变？这些可以在给定的数据集中计算。我们也可能需要确定结果和后续目标之间的相关性。特征相关性可以帮助构造特征，缺失值补全和纠错。
- **Converting**：在建模阶段，需要准备数据。和模型算法有关，有的需要把所有特征的值都转化为数值型的。因此需要把文本数据或者序号数据都转换成数值性。
- **Completing**：缺失值补全。
- **Correcting**：分析给定数据集中错误或者不准确的值，并改正或者直接删除异常样本。一个方法是检测样本或者特征的异常。如果一个特征对结果没有贡献，可以删除这个特征。
- **Creating**：根据一个或者一组特征构造新特征，依据是相关性、变换或者使目标完整。
- **Charting**：根据数据和解决目标来选择图表。

# 导包

In [ ]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

# 导入数据

In [ ]:
train_df = pd.read_csv('../input/train.csv')
test_df = pd.read_csv('../input/test.csv')
combine = [train_df, test_df]

# 通过数据描述来分析
## 数据集中有哪些特征？

In [ ]:
print(train_df.columns.values)

## 类别特征
把样本分成几个相似样本的集合。这些特征的值可以是名字、序数、百分比、按间隔划分。这个可以帮助我们选择合适的图进行可视化。
- Survived, Sex, and Embarked. Ordinal: Pclass

## 数值特征
每个样本的值基本不同，非连续的、连续的、基于时间序列的。
- Age, Fare. Discrete: SibSp, Parch

In [ ]:
# preview the data
train_df.head()

## 混合类型的特征
在同一个特征中存在数值型和数字字母型值。一般都是需要纠错的。
- Ticket是数值型和数字字母型的混合， Cabin是数字字母型的

## 包含错误的特征
大数据集很难来预览，小数据可以方便的预览一些样本来看哪些是需要纠错的。
- Name特征可能有错笔误，所以可以提取头衔、圆括号和引号作为 替代，或者短名来纠错

In [ ]:
train_df.tail()

## 特征的缺失处理
- Cabin > Age > Embarked ，训练集中缺失的数量排序
- Cabin > Age 测试集中的缺失

## 不同特征的数据类型
可以帮助我们转换数据
- 7个特征是整型或者浮点型，测试集中有6个
- 5个特征是字符串型

In [ ]:
train_df.info()
print('_'*40)
test_df.info()

## 样本中连续样本的分布
这有助于我们确定除了其他早期见解之外，实际问题领域的培训数据集的代表性如何。

- 总共891个样本，占Titanic样本(2224)中的41%
- Survived 是一个0-1的类别特征
- 样本中存活率为38%，实际是32%
- 大多数乘客（75%）没有父母和小孩在船上
- 大约30%的乘客有兄弟或者配偶在船上
- 只有小于1%的乘客的船费大于512美元
- 35-80岁的乘客小于1%

In [ ]:
train_df.describe()
# Review survived rate using `percentiles=[.61, .62]` knowing our problem description mentions 38% survival rate.
# Review Parch distribution using `percentiles=[.75, .8]`
# SibSp distribution `[.68, .69]`
# Age and Fare `[.1, .2, .3, .4, .5, .6, .7, .8, .9, .99]`

## 类别特征的分布

- Name在数据集中的每一个值都是唯一的(count=unique=891)
- Sex变量有两个可能的值，male占65%(top=male, freq=577/count=891)
- Cabin有一些样本的值是一样的，可能是有一些乘客共用一个船舱
- Embarked有3个值，大多数乘客都是S(top=S)
- Ticket有22%的重复率(unique=681)

In [ ]:
train_df.describe(include=['O'])

## 根据数据分析的假设
基于前面的数据分析做一些假设，可能会在后面进行验证

- Correlating
  - 我们需要知道各个特征和结果的相关关系。先在工程中做一下，然后在模型中计算一下。
- Completing
  1. 根据和存活率的相关性来补全Age缺失值
  2. 根据和存活率或者其他重要特征的相关性来补全Embarked缺失值
- Correcting
